# Обучение модели

## 1. Загрузка библиотек

In [1]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import statistics
from scipy import stats

import os

import pandas as pd
import numpy as np
import random
from scipy import stats
import math
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error,  accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression, load_boston, load_iris
from sklearn import linear_model

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, \
StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet,\
RidgeClassifier, LogisticRegressionCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score, accuracy_score, \
roc_auc_score, classification_report, r2_score, precision_score, recall_score, \
log_loss, mean_squared_log_error

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz 
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve

from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, \
    f1_score, log_loss, classification_report, roc_curve, auc, roc_auc_score, r2_score, \
    mean_squared_log_error

from tqdm.notebook import tqdm

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore")
simplefilter("ignore", category=RuntimeWarning)

RAND = 10
N_FOLDS = 5

## 2. Загрузка подготовленного датасета

In [2]:
df = pd.read_csv('df_train.csv', sep=',')

## 3. Catboost

### 3.1. Catboost baseline

In [3]:
X = df.drop(['Sample_ID', "Label"], axis=1)
y = df["Label"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=RAND)

X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)

In [5]:
cat_features = X_val.select_dtypes('category').columns.tolist()

clfcat = CatBoostClassifier(scale_pos_weight=y.agg(lambda x: x.eq(0).sum()) /
                            y.agg(lambda x: x.eq(1).sum()),
                            random_state=RAND,
                            eval_metric="F1",
                            cat_features=cat_features)
eval_set = [(X_val, y_val)]

clfcat.fit(X_train_,
           y_train_,
           eval_set=eval_set,
           early_stopping_rounds=100,
           verbose=False)

y_pred_cat = clfcat.predict(X_test)

In [6]:
def get_metrics(y_test, y_pred, y_score, name):
    df_metrics = pd.DataFrame()
    
    df_metrics['model'] = [name]
    
    # Основные метрики для задачи классификации
    df_metrics['Accuracy'] = [accuracy_score(y_test, y_pred)]
    
    # В ROC-AUC подаем фактические значения y и вероятности!!!!
    df_metrics['ROC_AUC'] = [roc_auc_score(y_test, y_score[:,1])]
    df_metrics['Precision'] = [precision_score(y_test, y_pred)]
    df_metrics['Recall'] = [recall_score(y_test, y_pred)]
    df_metrics['f1'] = [f1_score(y_test, y_pred)]
    df_metrics['Logloss'] = [log_loss(y_test, y_score)]
    
    return df_metrics

In [7]:
y_score = clfcat.predict_proba(X_test)

metrics = get_metrics(y_test,
                      y_pred_cat,
                      y_score,
                      name='Catboost_baseline_test')
metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.69465,0.368973


In [8]:
y_score_train = clfcat.predict_proba(X_train)
y_pred_cat_train = clfcat.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='Catboost_baseline_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235


### 3.2. Catboost optuna

In [10]:
from catboost import Pool

cat_feat = X_train.select_dtypes('category').columns.tolist()


def objective_cat_first(trial, X, y, N_FOLDS, random_state, cat_feat):
    params = {
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate",
                                             0.001,
                                             0.5,
                                             log=True),
        "random_state": RAND
    }

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        train_data = Pool(data=X_train, label=y_train, cat_features=cat_feat)
        eval_data = Pool(data=X_test, label=y_test, cat_features=cat_feat)

        model = CatBoostClassifier(**params)
        model.fit(train_data,
                  eval_set=eval_data,
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [11]:
import optuna

study_cat = optuna.create_study(direction="maximize", study_name="Cat_02")
func = lambda trial: objective_cat_first(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND, cat_feat=cat_feat)
# n_trials - кол-во итераций
study_cat.optimize(func, n_trials=10, show_progress_bar=True)

[I 2023-04-09 17:34:44,850] A new study created in memory with name: Cat_02


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-04-09 17:38:10,534] Trial 0 finished with value: 0.5952365049549123 and parameters: {'n_estimators': 1000, 'learning_rate': 0.0018816647674678845}. Best is trial 0 with value: 0.5952365049549123.
[I 2023-04-09 17:41:35,747] Trial 1 finished with value: 0.617015775742521 and parameters: {'n_estimators': 1000, 'learning_rate': 0.002678396898928292}. Best is trial 1 with value: 0.617015775742521.
[I 2023-04-09 17:44:49,623] Trial 2 finished with value: 0.6876450835177065 and parameters: {'n_estimators': 1000, 'learning_rate': 0.022851968752659508}. Best is trial 2 with value: 0.6876450835177065.
[I 2023-04-09 17:47:27,596] Trial 3 finished with value: 0.6958855537338493 and parameters: {'n_estimators': 1000, 'learning_rate': 0.036550518398102826}. Best is trial 3 with value: 0.6958855537338493.
[I 2023-04-09 17:50:51,446] Trial 4 finished with value: 0.6254295501474308 and parameters: {'n_estimators': 1000, 'learning_rate': 0.003144360092693803}. Best is trial 3 with value: 0.6958

In [12]:
def objective_cat_second(trial, X, y, N_FOLDS, random_state, cat_feat):
    params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_categorical("learning_rate",
                                  [0.09121441559229158]),  # 0.07
        "max_depth":
        trial.suggest_int("max_depth", 3, 12),
        "l2_leaf_reg":
        trial.suggest_uniform("l2_leaf_reg", 1e-5, 1e2),
        "random_strength":
        trial.suggest_uniform("random_strength", 10, 50),
        "border_count":
        trial.suggest_categorical("border_count", [128, 254]),
        # "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS", "No"]),
        # "grow_policy":
        # trial.suggest_categorical("grow_policy",
        #                        ["SymmetricTree", "Depthwise", "Lossguide"]),
        # "od_wait":
        # trial.suggest_int("od_wait", 500, 2000),
        # "leaf_estimation_iterations":
        #trial.suggest_int('leaf_estimation_iterations', 1, 15),
        # "cat_features": trial.suggest_categorical("cat_features", ["cat_features"]),
        "use_best_model":
        trial.suggest_categorical("use_best_model", [True]),
         "eval_metric":
         trial.suggest_categorical("eval_metric", ["F1"]),
        "loss_function":
        trial.suggest_categorical("loss_function", ["Logloss"]),
        # "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        # "class_weights": trial.suggest_categorical("class_weights", [1]),
        "scale_pos_weight":
        trial.suggest_categorical(
            "scale_pos_weight",
            [y.agg(lambda x: x.eq(0).sum()) / y.agg(lambda x: x.eq(1).sum())]),
        "random_state":
        RAND
    }

    # if params["bootstrap_type"] == "Bayesian":
    #  params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    # elif params["bootstrap_type"] == "Bernoulli":
    #  params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    # if params["grow_policy"] == "Lossguide":
    # params["num_leaves"] = trial.suggest_int("num_leaves", 20, 1000)

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        train_data = Pool(data=X_train, label=y_train, cat_features=cat_feat)
        eval_data = Pool(data=X_test, label=y_test, cat_features=cat_feat)

        model = CatBoostClassifier(**params)
        model.fit(train_data,
                  eval_set=eval_data,
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [13]:
study_cat = optuna.create_study(direction="maximize", study_name="Cat_02")
func = lambda trial: objective_cat_second(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND, cat_feat=cat_feat)
# n_trials - кол-во итераций
study_cat.optimize(func, n_trials=10, show_progress_bar=True)

[I 2023-04-09 18:02:45,876] A new study created in memory with name: Cat_02


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-04-09 18:06:17,931] Trial 0 finished with value: 0.712628944763741 and parameters: {'n_estimators': 1000, 'learning_rate': 0.09121441559229158, 'max_depth': 9, 'l2_leaf_reg': 27.26747129370398, 'random_strength': 38.18860899587888, 'border_count': 128, 'use_best_model': True, 'eval_metric': 'F1', 'loss_function': 'Logloss', 'scale_pos_weight': 2.999539806718822}. Best is trial 0 with value: 0.712628944763741.
[I 2023-04-09 18:07:22,517] Trial 1 finished with value: 0.7057972163315167 and parameters: {'n_estimators': 1000, 'learning_rate': 0.09121441559229158, 'max_depth': 6, 'l2_leaf_reg': 93.6179366054711, 'random_strength': 42.36624297809544, 'border_count': 254, 'use_best_model': True, 'eval_metric': 'F1', 'loss_function': 'Logloss', 'scale_pos_weight': 2.999539806718822}. Best is trial 0 with value: 0.712628944763741.
[I 2023-04-09 18:08:04,712] Trial 2 finished with value: 0.704469423609549 and parameters: {'n_estimators': 1000, 'learning_rate': 0.09121441559229158, 'max_d

In [14]:
study_cat.best_params

{'n_estimators': 1000,
 'learning_rate': 0.09121441559229158,
 'max_depth': 9,
 'l2_leaf_reg': 27.26747129370398,
 'random_strength': 38.18860899587888,
 'border_count': 128,
 'use_best_model': True,
 'eval_metric': 'F1',
 'loss_function': 'Logloss',
 'scale_pos_weight': 2.999539806718822}

In [15]:
cat_optuna = CatBoostClassifier(**study_cat.best_params)
cat_optuna.fit(X_train_,
               y_train_,
               cat_features=cat_features,
               eval_set=eval_set,
               verbose=False,
               early_stopping_rounds=100)

y_pred = cat_optuna.predict(X_test)
y_score = cat_optuna.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=y_pred,
                y_score=y_score,
                name='CatBoost_Optuna_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925


In [16]:
y_score_train = cat_optuna.predict_proba(X_train)
y_pred_cat_train = cat_optuna.predict(X_train)

metrics =metrics.append(get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='CatBoost_Optuna_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365


### 3.3. Catboost holdout

In [18]:
meta_X = pd.DataFrame()
meta_X_test = pd.DataFrame()

cv = StratifiedKFold(n_splits=N_FOLDS)

score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[train_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[train_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = CatBoostClassifier(**study_cat.best_params)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                silent=True,
                cat_features=cat_features,
                early_stopping_rounds=100)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
meta_X['cat_hold'] = model.predict_proba(X_train)[:,1]
meta_X_test['cat_hold'] = model.predict_proba(X_test)[:,1]

Fold: 1 f1_score 1.000
---
Fold: 2 f1_score 1.000
---
Fold: 3 f1_score 1.000
---
Fold: 4 f1_score 1.000
---
Fold: 5 f1_score 1.000
---


In [19]:
meta_X

,cat_hold
0,0.160316
1,0.037609
2,0.408851
3,0.003234
4,0.062173
...,...
8686,0.035886
8687,0.076527
8688,0.035998
8689,0.351413


In [20]:
meta_X_test

,cat_hold
0,0.379668
1,0.339881
2,0.003346
3,0.960385
4,0.150997
...,...
2168,0.010506
2169,0.844171
2170,0.009406
2171,0.419758


In [21]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)

In [22]:
from scipy import stats

test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

# oof - out-of-fold  - предсказания/значение метрики при обучении модели
# в ходе перекрестной проверки на validation data
print('F1 mean OOF: %.3f, std: %.3f' %
      (np.mean(score_oof), np.std(score_oof)))

# holdout  - предсказания/значение метрики при обучении модели
# в ходе перекрестной проверки на holdout data
print('F1 HOLDOUT: %.3f' % f1_score(y_test, test_pred))

F1 mean OOF: 1.000, std: 0.000
F1 HOLDOUT: 0.711


In [23]:
metrics=metrics.append(get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='CatBoost_Optuna_Holdout_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991


In [24]:
train_pred = model.predict(X_train)
train_pred_proba = model.predict_proba(X_train)

metrics=metrics.append(get_metrics(y_test=y_train,
                y_pred=train_pred,
                y_score=train_pred_proba,
                name='CatBoost_Optuna_Holdout_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649


## 4. RandomForest holdout+optuna

In [25]:
rf = RandomForestClassifier(random_state=RAND, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=10)

In [26]:
X_bin = pd.get_dummies(X, drop_first=True)
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=RAND)



def objective_rf(trial, X, y, N_FOLDS, rand):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "criterion": trial.suggest_categorical("criterion", ['gini', 'entropy', 'log_loss']),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        'bootstrap': trial.suggest_categorical("bootstrap", [True]),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'max_samples': trial.suggest_float('max_samples', 0.3, 0.99),
        'max_features': trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2']),
        'class_weight': trial.suggest_categorical("class_weight", ['balanced']),
        #'random_state': [rand] 
    }
    
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=rand)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = RandomForestClassifier(**params, n_jobs=-1, random_state=rand)
        model.fit(X_train, y_train)
        
        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [27]:
study_rf = optuna.create_study(direction="maximize", study_name="RF_01")
func = lambda trial: objective_rf(trial, X_train_bin, y_train, N_FOLDS, rand=RAND)
# n_trials - кол-во итераций
study_rf.optimize(func, n_trials=20, show_progress_bar=True)

[I 2023-04-09 18:39:43,735] A new study created in memory with name: RF_01


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-04-09 18:40:05,140] Trial 0 finished with value: 0.683869179177053 and parameters: {'n_estimators': 252, 'criterion': 'entropy', 'max_depth': 15, 'bootstrap': True, 'min_samples_leaf': 10, 'min_samples_split': 4, 'max_samples': 0.7234676071633888, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.683869179177053.
[I 2023-04-09 18:40:28,287] Trial 1 finished with value: 0.6689859747442888 and parameters: {'n_estimators': 397, 'criterion': 'gini', 'max_depth': 9, 'bootstrap': True, 'min_samples_leaf': 1, 'min_samples_split': 2, 'max_samples': 0.9477706782530015, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.683869179177053.
[I 2023-04-09 18:40:36,293] Trial 2 finished with value: 0.6642252896177501 and parameters: {'n_estimators': 427, 'criterion': 'gini', 'max_depth': 9, 'bootstrap': True, 'min_samples_leaf': 1, 'min_samples_split': 2, 'max_samples': 0.3469183661402568, 'max_features': 'log2', 'class_weight': 

In [28]:
pred_val = []
pred_score_val = []


cv = StratifiedKFold(n_splits=N_FOLDS)

for fold, (train_index, test_index) in enumerate(cv.split(X_train_bin, y_train)):
    X_train_, X_val = X_train_bin.iloc[train_index], X_train_bin.iloc[test_index]
    y_train_, y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    model = RandomForestClassifier(**study_rf.best_params, n_jobs=-1)
    model.fit(X_train_, y_train_)

    y_pred_val = model.predict(X_val)
    y_score_val = model.predict_proba(X_val)

    print("Fold:", fold + 1,
          "F1 SCORE %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # holdout list
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)

model.fit(X_train_bin, y_train)

meta_X['rf_01'] = np.concatenate(pred_score_val)[:,1]
meta_X_test['rf_01'] = model.predict_proba(X_test_bin)[:,1]

Fold: 1 F1 SCORE 0.690
---
Fold: 2 F1 SCORE 0.673
---
Fold: 3 F1 SCORE 0.688
---
Fold: 4 F1 SCORE 0.698
---
Fold: 5 F1 SCORE 0.680
---


In [29]:
meta_X_test

,cat_hold,rf_01
0,0.379668,0.500457
1,0.339881,0.606715
2,0.003346,0.028079
3,0.960385,0.835508
4,0.150997,0.288017
...,...,...
2168,0.010506,0.096891
2169,0.844171,0.739597
2170,0.009406,0.129803
2171,0.419758,0.196349


In [30]:
y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)

metrics = metrics.append(get_metrics(y_test, y_pred, y_score, 
                                     name='RandomForestClassifier_test'))
metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396


In [31]:
y_score_train = rf.predict_proba(X_train)
y_pred_cat_train = rf.predict(X_train)

metrics =metrics.append(get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='RandomForestClassifier_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408


In [32]:
meta_X_test

,cat_hold,rf_01
0,0.379668,0.500457
1,0.339881,0.606715
2,0.003346,0.028079
3,0.960385,0.835508
4,0.150997,0.288017
...,...,...
2168,0.010506,0.096891
2169,0.844171,0.739597
2170,0.009406,0.129803
2171,0.419758,0.196349


## 5. LGBM

### 5.1. LGBM baseline

In [33]:
clfLGBM = LGBMClassifier(scale_pos_weight=3, random_state=RAND)
eval_set = [(X_val, y_val)]

clfLGBM.fit(X_train_,
        y_train_,
        eval_metric="F1",
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=2)

y_pred = clfLGBM.predict(X_test)

[2]	valid_0's binary_logloss: 0.503283
[4]	valid_0's binary_logloss: 0.470297
[6]	valid_0's binary_logloss: 0.449358
[8]	valid_0's binary_logloss: 0.434769
[10]	valid_0's binary_logloss: 0.4254
[12]	valid_0's binary_logloss: 0.417275
[14]	valid_0's binary_logloss: 0.410716
[16]	valid_0's binary_logloss: 0.404976
[18]	valid_0's binary_logloss: 0.400721
[20]	valid_0's binary_logloss: 0.398287
[22]	valid_0's binary_logloss: 0.395194
[24]	valid_0's binary_logloss: 0.393001
[26]	valid_0's binary_logloss: 0.38942
[28]	valid_0's binary_logloss: 0.38697
[30]	valid_0's binary_logloss: 0.383908
[32]	valid_0's binary_logloss: 0.382414
[34]	valid_0's binary_logloss: 0.380389
[36]	valid_0's binary_logloss: 0.378361
[38]	valid_0's binary_logloss: 0.375188
[40]	valid_0's binary_logloss: 0.373925
[42]	valid_0's binary_logloss: 0.371784
[44]	valid_0's binary_logloss: 0.371699
[46]	valid_0's binary_logloss: 0.369886
[48]	valid_0's binary_logloss: 0.367739
[50]	valid_0's binary_logloss: 0.365253
[52]	val

In [34]:
y_pred = clfLGBM.predict(X_test)
y_score = clfLGBM.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=y_pred,
                y_score=y_score,
                name='LGBMboost_eval_baseline_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999


In [35]:
y_score_train = clfLGBM.predict_proba(X_train)
y_pred_cat_train = clfLGBM.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBMboost_eval_baseline_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


### 5.2. LGBM Optuna

In [36]:
def objective_lgb_first(trial, X, y, N_FOLDS, random_state=10):
    lgb_params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
    }

    N_FOLDS = 3
    cv = KFold(n_splits=N_FOLDS, shuffle=True)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # стрижка "на лету"
        # observation_key - Оценочная метрика для обрезки
        model = LGBMClassifier(**lgb_params)
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric="F1",
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [37]:
study = optuna.create_study(direction="maximize", study_name="LGB_21")


def func(trial): return objective_lgb_first(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND)


# n_trials - кол-во итераций
study.optimize(func, n_trials=20, show_progress_bar=True)

[I 2023-04-09 18:46:01,796] A new study created in memory with name: LGB_21


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-04-09 18:46:43,176] Trial 0 finished with value: 0.5924731157695688 and parameters: {'n_estimators': 1000, 'learning_rate': 0.0016830433862816147}. Best is trial 0 with value: 0.5924731157695688.
[I 2023-04-09 18:46:49,348] Trial 1 finished with value: 0.6875988846952173 and parameters: {'n_estimators': 1000, 'learning_rate': 0.14072480659549697}. Best is trial 1 with value: 0.6875988846952173.
[I 2023-04-09 18:46:54,579] Trial 2 finished with value: 0.6575744300837865 and parameters: {'n_estimators': 1000, 'learning_rate': 0.22997205940434293}. Best is trial 1 with value: 0.6875988846952173.
[I 2023-04-09 18:47:06,030] Trial 3 finished with value: 0.6848524824646978 and parameters: {'n_estimators': 1000, 'learning_rate': 0.036819725787581525}. Best is trial 1 with value: 0.6875988846952173.
[I 2023-04-09 18:47:25,653] Trial 4 finished with value: 0.69226165072675 and parameters: {'n_estimators': 1000, 'learning_rate': 0.01902909415998778}. Best is trial 4 with value: 0.6922616

In [38]:
def objective_lgb_second(trial, X, y, N_FOLDS, random_state=10):
    lgb_params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate":
        trial.suggest_categorical("learning_rate", [0.013607708322422411]),
        "num_leaves":
        trial.suggest_int("num_leaves", 20, 1000, step=20),
        "max_depth":
        trial.suggest_int("max_depth", 3, 12),
        "lambda_l1":
        trial.suggest_int("lambda_l1", 0, 100),
        "lambda_l2":
        trial.suggest_int("lambda_l2", 0, 100),
        "min_gain_to_split":
        trial.suggest_int("min_gain_to_split", 0, 20),
        "bagging_fraction":
        trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "bagging_freq":
        trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction":
        trial.suggest_float("feature_fraction", 0.2, 1.0),
        # "objective": trial.suggest_categorical("objective", ["mae"]),
        "random_state":
        RAND
    }

    N_FOLDS = 3
    cv = KFold(n_splits=N_FOLDS, shuffle=True)

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # стрижка "на лету"
        # observation_key - Оценочная метрика для обрезки
        pruning_callback = optuna.integration.LightGBMPruningCallback(
            trial, "F1")
        model = LGBMClassifier(**lgb_params)
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric="F1",
                  early_stopping_rounds=100,
                  verbose=0)

        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [39]:
study = optuna.create_study(direction="maximize", study_name="LGB_22")
func = lambda trial: objective_lgb_second(
    trial, X_train, y_train, N_FOLDS=N_FOLDS, random_state=RAND)
# n_trials - кол-во итераций
study.optimize(func, n_trials=20, show_progress_bar=True)

[I 2023-04-09 18:52:16,763] A new study created in memory with name: LGB_22


  0%|          | 0/20 [00:00<?, ?it/s]

[LightGBM] [Warning] feature_fraction is set=0.9018658491397653, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9018658491397653
[LightGBM] [Warning] min_gain_to_split is set=18, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=18
[LightGBM] [Warning] bagging_fraction is set=0.6977779593748604, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6977779593748604
[LightGBM] [Warning] lambda_l1 is set=28, reg_alpha=0.0 will be ignored. Current value: lambda_l1=28
[LightGBM] [Warning] lambda_l2 is set=37, reg_lambda=0.0 will be ignored. Current value: lambda_l2=37
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9018658491397653, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9018658491397653
[LightGBM] [Warning] min_gain_to_split is set=18, min_split_gain=0.0 will be ignored. Current value: min_gain_to

[LightGBM] [Warning] feature_fraction is set=0.29946378731384543, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.29946378731384543
[LightGBM] [Warning] min_gain_to_split is set=9, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9
[LightGBM] [Warning] bagging_fraction is set=0.20996825940700037, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20996825940700037
[LightGBM] [Warning] lambda_l1 is set=56, reg_alpha=0.0 will be ignored. Current value: lambda_l1=56
[LightGBM] [Warning] lambda_l2 is set=59, reg_lambda=0.0 will be ignored. Current value: lambda_l2=59
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.29946378731384543, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.29946378731384543
[LightGBM] [Warning] min_gain_to_split is set=9, min_split_gain=0.0 will be ignored. Current value: min_gain

[LightGBM] [Warning] feature_fraction is set=0.37374089012941564, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.37374089012941564
[LightGBM] [Warning] min_gain_to_split is set=2, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2
[LightGBM] [Warning] bagging_fraction is set=0.7659772112298304, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7659772112298304
[LightGBM] [Warning] lambda_l1 is set=67, reg_alpha=0.0 will be ignored. Current value: lambda_l1=67
[LightGBM] [Warning] lambda_l2 is set=24, reg_lambda=0.0 will be ignored. Current value: lambda_l2=24
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[I 2023-04-09 18:52:34,393] Trial 6 finished with value: 0.5203186780026939 and parameters: {'n_estimators': 1000, 'learning_rate': 0.013607708322422411, 'num_leaves': 1000, 'max_depth': 3, 'lambda_l1': 67, 'lambda_l2': 24, 'min_gain_to_split': 2, 'bagging_fraction'

[I 2023-04-09 18:52:44,624] Trial 9 finished with value: 0.5014788370591702 and parameters: {'n_estimators': 1000, 'learning_rate': 0.013607708322422411, 'num_leaves': 480, 'max_depth': 9, 'lambda_l1': 37, 'lambda_l2': 53, 'min_gain_to_split': 19, 'bagging_fraction': 0.7878758386519662, 'bagging_freq': 1, 'feature_fraction': 0.6556147982899577}. Best is trial 5 with value: 0.6379557521014103.
[LightGBM] [Warning] feature_fraction is set=0.5164315253396952, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5164315253396952
[LightGBM] [Warning] min_gain_to_split is set=13, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=13
[LightGBM] [Warning] bagging_fraction is set=0.5222176750901297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5222176750901297
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current v

[LightGBM] [Warning] feature_fraction is set=0.5051726260544819, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5051726260544819
[LightGBM] [Warning] min_gain_to_split is set=9, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=9
[LightGBM] [Warning] bagging_fraction is set=0.9244077257262697, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9244077257262697
[LightGBM] [Warning] lambda_l1 is set=14, reg_alpha=0.0 will be ignored. Current value: lambda_l1=14
[LightGBM] [Warning] lambda_l2 is set=16, reg_lambda=0.0 will be ignored. Current value: lambda_l2=16
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.5051726260544819, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5051726260544819
[LightGBM] [Warning] min_gain_to_split is set=9, min_split_gain=0.0 will be ignored. Current value: min_gain_to_sp

[LightGBM] [Warning] feature_fraction is set=0.20377016437136067, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.20377016437136067
[LightGBM] [Warning] min_gain_to_split is set=11, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11
[LightGBM] [Warning] bagging_fraction is set=0.6044941532074191, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6044941532074191
[LightGBM] [Warning] lambda_l1 is set=21, reg_alpha=0.0 will be ignored. Current value: lambda_l1=21
[LightGBM] [Warning] lambda_l2 is set=75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=75
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[I 2023-04-09 18:53:02,634] Trial 16 finished with value: 0.5481487679865039 and parameters: {'n_estimators': 1000, 'learning_rate': 0.013607708322422411, 'num_leaves': 780, 'max_depth': 3, 'lambda_l1': 21, 'lambda_l2': 75, 'min_gain_to_split': 11, 'bagging_fracti

[I 2023-04-09 18:53:10,158] Trial 19 finished with value: 0.5909058715551977 and parameters: {'n_estimators': 1000, 'learning_rate': 0.013607708322422411, 'num_leaves': 400, 'max_depth': 4, 'lambda_l1': 9, 'lambda_l2': 21, 'min_gain_to_split': 11, 'bagging_fraction': 0.37044687516224384, 'bagging_freq': 1, 'feature_fraction': 0.7083861280458134}. Best is trial 10 with value: 0.6524785070772621.


In [40]:
lgb_grid = LGBMClassifier(**study.best_params)
lgb_grid.fit(X_train_,
             y_train_,
             eval_metric="F1",
             eval_set=eval_set,
             verbose=False,
             early_stopping_rounds=100)

y_pred = lgb_grid.predict(X_test)
y_score = lgb_grid.predict_proba(X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test.values,
                y_pred=y_pred,
                y_score=y_score,
                name='LGBoost_Optuna_test'))
metrics

[LightGBM] [Warning] feature_fraction is set=0.5164315253396952, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5164315253396952
[LightGBM] [Warning] min_gain_to_split is set=13, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=13
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.5222176750901297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5222176750901297
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


In [41]:
y_score_train = lgb_grid.predict_proba(X_train)
y_pred_cat_train = lgb_grid.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBoost_Optuna_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


### 5.3. LGBM Holdout

In [42]:
pred_val = []
pred_score_val = []

cv = StratifiedKFold(n_splits=N_FOLDS)
for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

    model = LGBMClassifier(is_unbalance=True, random_state=RAND)
    model.fit(X_train_,
              y_train_,
              eval_set=[(X_val, y_val)],
              eval_metric="F1",
              early_stopping_rounds=100,
              verbose=0)

    y_pred_val = model.predict(X_val)
    y_score_val = model.predict_proba(X_val)

    print("Fold:", fold + 1,
          "F1 SCORE %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # holdout list
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)

model.fit(X_train, y_train)

meta_X['lgb_01'] = np.concatenate(pred_score_val)[:, 1]
meta_X_test['lgb_01'] = model.predict_proba(X_test)[:, 1]

Fold: 1 F1 SCORE 0.715
---
Fold: 2 F1 SCORE 0.709
---
Fold: 3 F1 SCORE 0.709
---
Fold: 4 F1 SCORE 0.709
---
Fold: 5 F1 SCORE 0.715
---


In [43]:
score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
    X_train_, X_val = X_train.iloc[train_idx], X_train.iloc[train_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[train_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = LGBMClassifier(**study.best_params)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                eval_metric="F1",
                early_stopping_rounds=100,
                verbose=0)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
    
meta_X['lgb_02'] = model.predict_proba(X_train)[:, 1]
meta_X_test['lgb_02'] = model.predict_proba(X_test)[:, 1]

[LightGBM] [Warning] feature_fraction is set=0.5164315253396952, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5164315253396952
[LightGBM] [Warning] min_gain_to_split is set=13, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=13
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.5222176750901297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5222176750901297
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Fold: 1 f1_score 0.691
---
[LightGBM] [Warning] feature_fraction is set=0.5164315253396952, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5164315253396952
[LightGBM] [Warning] min_gain_to_split is set=13, min_split_gain=0.0 will be ignored. Cur

In [44]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)
test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

In [45]:
meta_X

,cat_hold,rf_01,lgb_01,lgb_02
0,0.160316,0.725853,0.643648,0.630835
1,0.037609,0.538449,0.083909,0.293180
2,0.408851,0.798530,0.894912,0.802514
3,0.003234,0.084115,0.007239,0.035794
4,0.062173,0.442030,0.463000,0.441411
...,...,...,...,...
8686,0.035886,0.167762,0.062253,0.056978
8687,0.076527,0.202261,0.120534,0.109937
8688,0.035998,0.114443,0.056594,0.038630
8689,0.351413,0.354387,0.352529,0.276506


In [46]:
meta_X_test

,cat_hold,rf_01,lgb_01,lgb_02
0,0.379668,0.500457,0.559945,0.316464
1,0.339881,0.606715,0.698389,0.574733
2,0.003346,0.028079,0.022153,0.023572
3,0.960385,0.835508,0.912270,0.716855
4,0.150997,0.288017,0.134618,0.118043
...,...,...,...,...
2168,0.010506,0.096891,0.070634,0.031913
2169,0.844171,0.739597,0.544917,0.559857
2170,0.009406,0.129803,0.064475,0.053312
2171,0.419758,0.196349,0.350820,0.278632


In [47]:
metrics = metrics.append(
    get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='LGBM_Holdout_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


In [48]:
y_score_train = model.predict_proba(X_train)
y_pred_cat_train = model.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='LGBM_Holdout_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


## 6. xgboost holdout

In [49]:
import xgboost as xgb

score_oof = []
predictions_test = []
finish_test_preds_proba = []
pred_val = []
pred_score_val = []
early_stop=True

for fold, (train_idx, test_idx) in enumerate(cv.split(X_train_bin, y_train)):
    X_train_, X_val = X_train_bin.iloc[train_idx], X_train_bin.iloc[test_idx]
    y_train_, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]
    y_val_exp = np.exp(y_val) - 1

    
    model = XGBClassifier(scale_pos_weight=3, random_state=RAND)

    model.fit(X_train_,
                y_train_,
                eval_set=[(X_val, y_val)],
                eval_metric="logloss",
                early_stopping_rounds=100,
                verbose=0)


    y_pred_val = model.predict(X_val)
    y_pred = model.predict(X_test)
    preds_test_proba = model.predict_proba(X_test)
    finish_test_preds_proba.append(preds_test_proba)

    print(
        "Fold:", fold + 1,
        "f1_score %.3f" % f1_score(y_val, y_pred_val))
    print("---")

    # oof list
    score_oof.append(f1_score(y_val, y_pred_val))
    # holdout list
    y_score_val = model.predict_proba(X_val)
    predictions_test.append(y_pred)
    pred_val.append(y_pred_val)
    pred_score_val.append(y_score_val)
    
meta_X['xgb_01'] = model.predict_proba(X_train)[:, 1]
meta_X_test['xgb_01'] = model.predict_proba(X_test)[:, 1]

Fold: 1 f1_score 0.707
---
Fold: 2 f1_score 0.690
---
Fold: 3 f1_score 0.696
---
Fold: 4 f1_score 0.707
---
Fold: 5 f1_score 0.696
---


In [50]:
test_pred_proba = np.mean(finish_test_preds_proba, axis=0)
test_pred = stats.mode(np.column_stack(predictions_test), axis=1)[0]

In [51]:
metrics=metrics.append(get_metrics(y_test=y_test,
                y_pred=test_pred,
                y_score=test_pred_proba,
                name='XGB_Holdout_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


In [52]:
y_score_train = model.predict_proba(X_train)
y_pred_cat_train = model.predict(X_train)

metrics = metrics.append(
    get_metrics(y_train,
                y_pred_cat_train,
                y_score_train,
                name='XGB_Holdout_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


## 7. Stacking

In [53]:
meta_X_test

,cat_hold,rf_01,lgb_01,lgb_02,xgb_01
0,0.379668,0.500457,0.559945,0.316464,0.299519
1,0.339881,0.606715,0.698389,0.574733,0.857833
2,0.003346,0.028079,0.022153,0.023572,0.007652
3,0.960385,0.835508,0.912270,0.716855,0.934065
4,0.150997,0.288017,0.134618,0.118043,0.030780
...,...,...,...,...,...
2168,0.010506,0.096891,0.070634,0.031913,0.000757
2169,0.844171,0.739597,0.544917,0.559857,0.547519
2170,0.009406,0.129803,0.064475,0.053312,0.011578
2171,0.419758,0.196349,0.350820,0.278632,0.567709


In [54]:
meta_X

,cat_hold,rf_01,lgb_01,lgb_02,xgb_01
0,0.160316,0.725853,0.643648,0.630835,0.214860
1,0.037609,0.538449,0.083909,0.293180,0.049497
2,0.408851,0.798530,0.894912,0.802514,0.776404
3,0.003234,0.084115,0.007239,0.035794,0.000248
4,0.062173,0.442030,0.463000,0.441411,0.164105
...,...,...,...,...,...
8686,0.035886,0.167762,0.062253,0.056978,0.085563
8687,0.076527,0.202261,0.120534,0.109937,0.073400
8688,0.035998,0.114443,0.056594,0.038630,0.064124
8689,0.351413,0.354387,0.352529,0.276506,0.268837


In [55]:
df_label_train = pd.DataFrame(data=y_train)
df_label_train = df_label_train.reset_index()
meta_X_final=pd.concat([meta_X, df_label_train], axis=1)
meta_X_final.drop(['index'], axis=1, inplace=True)
meta_X_final

,cat_hold,rf_01,lgb_01,lgb_02,xgb_01,Label
0,0.160316,0.725853,0.643648,0.630835,0.214860,0
1,0.037609,0.538449,0.083909,0.293180,0.049497,0
2,0.408851,0.798530,0.894912,0.802514,0.776404,0
3,0.003234,0.084115,0.007239,0.035794,0.000248,0
4,0.062173,0.442030,0.463000,0.441411,0.164105,0
...,...,...,...,...,...,...
8686,0.035886,0.167762,0.062253,0.056978,0.085563,0
8687,0.076527,0.202261,0.120534,0.109937,0.073400,0
8688,0.035998,0.114443,0.056594,0.038630,0.064124,0
8689,0.351413,0.354387,0.352529,0.276506,0.268837,0


In [56]:
y_train_final = meta_X_final['Label']
X_train_final = meta_X_final.drop(['Label'], axis=1)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_train_final,
                                                            y_train_final,
                                                            test_size=0.2,
                                                            stratify=y_train,
                                                            random_state=RAND)

X_train_, X_val, y_train_, y_val = train_test_split(X_train_f,
                                                    y_train_f,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)

clfcat = CatBoostClassifier(scale_pos_weight=3,
                            random_state=RAND,
                            eval_metric="F1")
eval_set = [(X_val, y_val)]

clfcat.fit(X_train_,
           y_train_,
           eval_set=eval_set,
           early_stopping_rounds=100,
           verbose=False)

y_pred = clfcat.predict(meta_X_test)

In [57]:
y_score = clfcat.predict_proba(meta_X_test)

metrics = metrics.append(
    get_metrics(y_test=y_test, y_pred=y_pred, y_score=y_score,
                name='Stacking_test'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


In [58]:
y_score = clfcat.predict_proba(meta_X)
y_pred = clfcat.predict(meta_X)

metrics = metrics.append(
    get_metrics(y_test=y_train, y_pred=y_pred, y_score=y_score,
                name='Stacking_train'))

metrics

,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,Catboost_baseline_test,0.829268,0.896079,0.627976,0.777164,0.694650,0.368973
0,Catboost_baseline_train,0.929582,0.981090,0.801934,0.953981,0.871375,0.218235
0,CatBoost_Optuna_test,0.834791,0.898543,0.645570,0.751381,0.694468,0.359925
0,CatBoost_Optuna_train,0.949028,0.987199,0.853350,0.961344,0.904133,0.191365
0,CatBoost_Optuna_Holdout_test,0.853198,0.904950,0.699288,0.723757,0.711312,0.336991
0,CatBoost_Optuna_Holdout_train,0.971235,0.988823,0.941663,0.943396,0.942529,0.111649
0,RandomForestClassifier_test,0.844455,0.883745,0.759494,0.552486,0.639659,0.375396
0,RandomForestClassifier_train,1.000000,1.000000,1.000000,1.000000,1.000000,0.099408
0,LGBMboost_eval_baseline_test,0.837552,0.897906,0.656766,0.732965,0.692776,0.355999
0,LGBMboost_eval_baseline_train,0.953630,0.984293,0.877882,0.946157,0.910742,0.162238


Выводы:
- 1) Лучшие показатели f1, roc_auc, logloss на тестовых данных выдал CatBoost_Optuna_Holdout 
- 2) Самые непереобученные модели: LGBoost_Optuna, LGBM_Holdout.
- 3) Stacking не показал хороших результатов на тестовых данных, но показал лучшие результаты, среди моих моделей, на скрытых данных, по которым и ставится оценка в соревновании
- 4) RandomForest показал одни и самых плохих результатов по основным метрикам, но лучший по Precision. Поэтому я оставил его в стэкинге.
- 5) Пробовал разное кол-во и состав моделей для стекинга. Самые лучшие метрики на скрытых данных получились у этих 5 моделей.
- 6) Метрики на тестовых данных и на скрытых данных не коррелировали между собой, поэтому обучение для соревнований проходило вслепую.

[11 место в соревновании](https://zindi.africa/competitions/landslide-prevention-and-innovation-challenge/leaderboard)